# CLRNET_ALL_train
### Import Python Packages


In [1]:
from tensorflow.python.keras.backend import clear_session
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model
import numpy as np
from resnet_convlstm import ResNet
from Utility_functions import create_sequence, FreezeBatchNormalization,AdditionalValidationSets
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime as dt
from DFVDSequence import DFVDSequence
import matplotlib.pyplot as plt
print("yolo")
from sklearn.metrics import classification_report
import pandas as pd
# import shutil
from tensorflow.python.keras import layers
# from src.xception_convlstm import Xception
# print(cv2.__version__)
# import random
# random.seed(32)

dataset_dir='DeepFakeDatasetReal'
# from src.cl_basic import cl_basic
import ipykernel
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import backend as K
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# Choose GPU NUMBERS [0, 1, 2, 3]
os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

## Load DF+FS Transfer Dataset

In [2]:
train_video_per_batch=40
val_video_per_batch=25
test_video_per_batch=25
train_frames_per_video=10 #total frames
val_frames_per_video=5 #total frames
test_frames_per_video=5 #total frames
frames_per_video_per_batch=5 #frames in one batch
image_size=128
channel=3
train_augumentation=True
val_augumentation=False
test_augumentation=False
X_train,y_train, class_weights_train = create_sequence(['datasets/FaceSwap/train/',
                                               'datasets/NeuralTextures/train/'],
                                                      frames_per_video_per_batch,
                                                       train_frames_per_video)

X_val_0,y_val_0,class_weights_val_0=create_sequence(['datasets/FaceSwap/val/',
                                                     'datasets/NeuralTextures/val/'],
                                              frames_per_video_per_batch,
                                                    val_frames_per_video)

train_it=DFVDSequence(X_train,y_train,train_video_per_batch,
                      frames_per_video_per_batch,image_size,
                      train_augumentation,True)

val_it=DFVDSequence(X_val_0,y_val_0,val_video_per_batch,
                    frames_per_video_per_batch,image_size,
                    val_augumentation,False)


fig=plt.figure(figsize=(10, 100))
columns = 4
rows = 23
print("Training")
x=1
temp=train_it.__getitem__(1)
fig.add_subplot(rows, columns, x)
print("Min:",np.array(temp[0][0,0]).min(),"Max:",np.array(temp[0][0,0]).max())
print("Min:",np.array(temp[0][1,0]).min(),"Max:",np.array(temp[0][1,0]).max())
plt.imshow(np.array(temp[0][0,0]))
fig.add_subplot(rows, columns, x+1)
plt.imshow(np.array(temp[0][1,0]))
print(temp[0].shape,temp[1].shape,train_it.dataset_size,train_it.frame_counter,train_it.__len__(),temp[1][0],temp[1][1])
# x+=2
plt.title("Training Set Examples")        
plt.show()
train_it.on_epoch_end()

fig=plt.figure(figsize=(10, 100))
columns = 4
rows = 23
print("Validation")
x=1
temp=val_it.__getitem__(1)
fig.add_subplot(rows, columns, x)
print("Min:",np.array(temp[0][0,0]).min(),"Max:",np.array(temp[0][0,0]).max())
print("Min:",np.array(temp[0][1,0]).min(),"Max:",np.array(temp[0][1,0]).max())
plt.imshow(np.array(temp[0][0,0]),)
fig.add_subplot(rows, columns, x+1)
plt.imshow(np.array(temp[0][1,0]))
print(temp[0].shape,temp[1].shape,val_it.dataset_size,val_it.frame_counter,val_it.__len__(),temp[1][0],temp[1][1])
plt.title("Validation Set Examples")   
plt.show()
val_it.on_epoch_end()


In [3]:
fig=plt.figure(figsize=(10, 100))
columns = 5
rows = 40
print("ALL")
x=1
temp=train_it.__getitem__(1)
fig.add_subplot(rows, columns, x)
plt.imshow(np.array(temp[0][0,0]))
fig.add_subplot(rows, columns, x+1)
plt.imshow(np.array(temp[0][0,1]))
fig.add_subplot(rows, columns, x+2)
plt.imshow(np.array(temp[0][0,2]))
fig.add_subplot(rows, columns, x+3)
plt.imshow(np.array(temp[0][0,3]))
fig.add_subplot(rows, columns, x+4)
plt.imshow(np.array(temp[0][0,4]))
print("Min:",np.array(temp[0][0,0]).min(),"Max:",np.array(temp[0][0,0]).max())
fig.add_subplot(rows, columns, x+5)
plt.imshow(np.array(temp[0][1,0]))
fig.add_subplot(rows, columns, x+6)
plt.imshow(np.array(temp[0][1,1]))
fig.add_subplot(rows, columns, x+7)
plt.imshow(np.array(temp[0][1,2]))
fig.add_subplot(rows, columns, x+8)
plt.imshow(np.array(temp[0][1,3]))
fig.add_subplot(rows, columns, x+9)
plt.imshow(np.array(temp[0][1,4]))
print("Min:",np.array(temp[0][1,0]).min(),"Max:",np.array(temp[0][1,0]).max())
print(temp[0].shape,temp[1].shape,train_it.dataset_size,train_it.frame_counter,train_it.__len__(),temp[1][0],temp[1][1])
plt.title("Training Set Examples")        
plt.show()
train_it.on_epoch_end()

# Load Model

In [4]:
model=ResNet(input_shape=(frames_per_video_per_batch,image_size, image_size, channel), classes=2, block='bottleneck', residual_unit='v2',
           repetitions=[3, 4, 6, 3], initial_filters=64, activation='softmax', include_top=False,
           input_tensor=None, dropout=0.25, transition_dilation_rate=(1, 1),
           initial_strides=(2, 2), initial_kernel_size=(7, 7), initial_pooling='max',
           final_pooling=None, top='classification')
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy',
                  optimizer=adam_fine,
                  metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)

# Compiling Model

In [5]:
model_checkpoint_callback = ModelCheckpoint(
    filepath="CLR_FS+NT_Ebest.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
csv_logger = CSVLogger("CLR_FS+NT_train.csv", append=True, separator=',')
callbacks_list = [csv_logger,model_checkpoint_callback] 

# Training

In [6]:
model.fit_generator(generator=train_it,
                    epochs=50,
                    validation_data=val_it,
                    callbacks=callbacks_list,
                    shuffle=False,
                    class_weight=class_weights_train,
                    verbose=1)
model.save_weights("CLR_FS+NT_E50.h5")

In [6]:
model.load_weights("CLR_FS+NT_E50.h5")

In [7]:
model.fit_generator(generator=train_it,
                    epochs=150,
                    validation_data=val_it,initial_epoch=50,
                    callbacks=callbacks_list,
                    shuffle=False,
                    class_weight=class_weights_train,
                    verbose=1)
model.save_weights("CLR_FS+NT_E150.h5")